In [5]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga de pacientes y empresas

In [6]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7920, 7)

In [8]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "poliza__c",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
    ]
]
Pacientes_SF.shape

KeyError: "['poliza__c'] not in index"

In [ ]:
# recuperar todos los pacientes ID desde Contact
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

In [ ]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa en account contact no es la q se ve
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

In [ ]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

********************************************************************************************************************************************************************

Aca las empresas que se ven de pacientes programas y programas por empresa


In [ ]:
# buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

In [ ]:
#unir empresas y todas sus caracteristicas programas pacientes


empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
empresas_unidas.shape

In [ ]:
empresas_unidas.head()

In [ ]:
uni_pacientes = Pacientes_SF[['Id', 'IdentificationId__pc','Name', 'Identification_Type__c', 'Ocupaci_n__c', 'Pa_s_del_Paciente__c', 'PersonTitle' ,'PersonDepartment']]

uni_empresas = empresas_unidas[['Paciente__c','Programas_por_Empresas__c','padre__c','Empresa__c','Id_y','Empresa_Padre__c','Nombre_Empresa__c','Name']]
pacientessf_empresas = pd.merge(
    left=uni_empresas,
    right=uni_pacientes,
    how="left",
    left_on="Paciente__c",
    right_on="Id",
)
pacientessf_empresas.shape

In [ ]:
pacientessf_empresas.head()


ACA tenemos toda la relacion de pacientes y empresas del sistema

In [ ]:
pacientessf_empresas = pd.merge(
    left=pacientessf_empresas,
    right=Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
pacientessf_empresas.shape

In [ ]:
pacientessf_empresas.head()

In [ ]:
#empresas_unidas.head()
#apprix_1 = empresas_unidas.iloc[:611178,:]
#apprix_2 = empresas_unidas.iloc[611178:,:]
#apprix_1.to_excel(
#    "C:/Users/jmartinez/Documents/empresasA.xlsx", index=False
#)
#apprix_2.to_excel(
#    "C:/Users/jmartinez/Documents/empresasB.xlsx", index=False
#)

Descarga desde Citas de Servicios

In [ ]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c'
        
    ]
]
Citas_de_servicio.shape


In [ ]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio2.shape


In [ ]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
    }
)

Gran_Citas_deservicio.shape

In [ ]:
Gran_Citas_deservicio.head()

Programas descargas

In [ ]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


In [ ]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)


ficha_nutricion.shape

In [ ]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)

ficha_Apssex.shape

In [ ]:
#Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Psicologia['clinica'] = 'Psicología'
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Psicologia.shape

In [ ]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Sueno.shape

In [ ]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Cronico.shape

In [ ]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_Telemedicina.shape

In [ ]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Veterinaria.shape

In [ ]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Orientacion.shape

In [ ]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Front.shape

In [ ]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica'
    }
)
ficha_general.shape


In [ ]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]

ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Ingreso_dep.shape

In [ ]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte,ficha_general], axis=0)
Clinicas.shape

In [ ]:
Clinicas.head()

In [ ]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

In [ ]:
Reortes_general.head()

In [ ]:
Reortes_general["Mes"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.month

Reortes_general["Año"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.year
Reortes_general["Dia"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime("%d-%m-%Y")

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    'Orientación Médica':'Orientación Médica' ,
    'Asesoría Deportiva':'Deporte',
    'Asistencia Emocional':"Psicología",
    'Asistencia Nutricional':'Nutrición',
    'Clínica APSSex':'Asesoría Psicologíca en Salud Sexual',
    'Clínica Del Sueño':'Clínica Del Sueño',
    'Clínica Veterinaria':'Veterinaria',
    

    
}
Reortes_general.replace({"Clinica__c": clinicas_}, inplace=True)


Reortes_general.head()



In [ ]:


Reortes_general['Fecha_chilena'] = Reortes_general['Fecha_chilena'].dt.tz_localize(None)

Reortes_general.Rut.fillna('0', inplace=True)
Reortes_general.to_excel(
    "C:/Users/jmartinez/Documents/reportes.xlsx", index=False
)

In [ ]:
pacientessf_empresas.head()

In [ ]:

unir_reporte_con_empresas = pd.merge(
    left=Reortes_general,
    right=pacientessf_empresas,
    how="left",
    left_on="Rut",
    right_on="IdentificationId__pc_x",
)
unir_reporte_con_empresas.Empleador.fillna(unir_reporte_con_empresas.Name , inplace=True)
unir_reporte_con_empresas.shape



In [ ]:
Reortes_general.shape

In [ ]:
unir_reporte_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes2.xlsx", index=False
)

In [ ]:
Metlife = unir_reporte_con_empresas[(unir_reporte_con_empresas['Campa_a__c']=='MetLife')]
Metlife.to_excel(
    "C:/Users/jmartinez/Documents/Metlife.xlsx", index=False
)

In [ ]:
nuevos = ("C:/Users/jmartinez/Documents/bechtel.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

bechtel = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    #right=pacientessf_empresas,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_SF

In [ ]:
bechtel.to_excel(
    "C:/Users/jmartinez/Documents/bechtel_salida2.xlsx", index=False
)